# About

This notebook downloads canopy height (10m resolution) data over Santa Barabara County  from the [California Forest Observatory (CFO) API](https://github.com/forestobservatory/cfo-api). To do this we follow the 'Custom Data Clipping' section in [this CFO API tutorial](https://github.com/forestobservatory/cfo-api/blob/master/demos/api-introduction.ipynb). A CFO account is needed to run the notebook. 

[Source](https://databasin.org/datasets/1cdc07adea7d4dee9cc1f07ab44cfef3/) of the Santa Barbara County shapefile. This was converted to a geojson file using geopandas.

**NOTEBOOK VARIABLES**: 
- `year` (list of int): years from which we will access the CFO California canopy height raster. Must be a subset of [2016, 2018, 2020] since CFO's canopy height data is not available for years before 2016. 

**OUTPUT**:
-  canopy height rasters for Santa Barbara County.






# Install California Forest Observatory API package if not available

In [1]:
#pip install cfo

In [2]:
import os

import cfo
from osgeo import gdal

# Specify year and file path for canopy height data

In [3]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# year should be a subset of [2016, 2018, 2020]
years = [2016, 2018, 2020]

# ***************************************************
# ***************************************************

# Authenticate 

In [5]:
forest_api = cfo.api()
forest_api.authenticate()galaz-garcia@nceas.ucsb.edu

CFO E-mail:  galaz-garcia@nceas.ucsb.edu
CFO Password:  ···········


200

# Download canopy height

In [7]:
# load SB shape
vector = os.path.join(os.getcwd(),'SantaBarbaraCounty_geojson','SantaBarbaraCounty.geojson')

for year in years:
    # get the 10m canopy height asset id for the given year
    search = forest_api.search(geography='California', metric='CanopyHeight', year=year, resolution=10)
    itemid = search[0]

    # and get the gdal file path reference
    input_file = forest_api.fetch(itemid, gdal=True)

    # set the output raster file
    output_file = os.path.join(os.getcwd(),'SantaBarbaraCounty_CanopyHeight', 'SantaBarbaraCounty_CanopyHeight_'+str(year)+'.tif')

    # set the gdalwarp options
    options = gdal.WarpOptions(
        creationOptions = ["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=YES", "NUM_THREADS=ALL_CPUS"],
        cutlineDSName = vector,
        cropToCutline = True
    )

    # and run the command
    warp = gdal.Warp(output_file, input_file, options=options)
    warp.FlushCache()